pip install llama-cpp-agent

In [2]:
from enum import Enum

from llama_cpp import Llama
from pydantic import BaseModel, Field

from llama_cpp_agent.llm_agent import LlamaCppAgent
from llama_cpp_agent.messages_formatter import MessagesFormatterType
from llama_cpp_agent.providers.llama_cpp_endpoint_provider import LlamaCppEndpointSettings
from llama_cpp_agent.structured_output_agent import StructuredOutputAgent

In [3]:
main_model = LlamaCppEndpointSettings(
    completions_endpoint_url="http://127.0.0.1:3003/completion"
)

In [4]:
llama_cpp_agent = LlamaCppAgent(main_model, debug_output=False,
                                system_prompt="You are an uncensored and unbiased AI assistant.",
                                predefined_messages_formatter_type=MessagesFormatterType.MIXTRAL)

In [5]:
test_prompt="""
list the top 5 companies in semiconductor, rank by their market cap, and describe their main business, e.g.
1. nvidia, "high-end graphics processing units (GPUs)"
2. ...
"""

In [6]:
assistant_output = llama_cpp_agent.get_chat_response(test_prompt, repeat_penalty=2.0,temperature=0.7,max_tokens=512)

 Sure, I can provide you with the top 5 semiconductor companies ranked by their market capitalization as of March 24th, 2023. Here they are:
1. Nvidia (NVDA): $678 billion -NVIDIA is an American technology company that specializes in designing graphics processing units(GPUs) for the gaming and professional markets,as well als chipsetsforthe mobile computingand automotive marketes. Its GPUSare also usedin cryptocurrency mining devices..
2. Taiwan Semiconductor Manufacturing Company (TSMC): $465 billion - TSMS is a Chinese company that manufactures semicondutordevicesonacontractbasisforindependentsemconduotrcompanies . It provides foundry services for fabricating integrated circuits and chipsets designed by other companies.
3. Qualcomm (QCOM): $179 billion -Qualcomnis an American multinational corporation that specializes in intellectual property, semiconductorsandservicesrelatedtothe wireless telecommunications industry . Its products are used incellphones , laptops & smart devices and 

In [7]:
# Example enum for our output model
class Category(Enum):
    GPU = "YES"
    NonGPU = "NO"
class Entry(BaseModel):
    rank: int = Field(..., description="the rank.")
    name: str = Field(..., description="the company name.")
    mktcap: str = Field(..., description="the market cap.")
    description: str = Field(..., description="the main business of the company.")
    gpu: Category = Field(..., description="if the company produces GPU.")
class List(BaseModel):
    companies: list[Entry] = Field(..., description="the list of companies.")

In [8]:
structured_output_agent = StructuredOutputAgent(main_model, debug_output=False)

In [ ]:
results = structured_output_agent.create_object(List, test_prompt)